In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## Auto-fixing parser

`OutputFixingParser` 는 다른 출력 구문 분석기를 래핑하고, 첫 번째 구문 분석기가 실패할 경우 다른 LLM을 호출하여 오류를 수정합니다.

하지만 에러를 던지는 것 외에도 다른 작업을 수행할 수 있습니다.

특히 형식이 잘못된 출력을 형식이 지정된 명령어와 함께 모델에 전달하여 수정하도록 요청할 수 있습니다.

이 예제에서는 위의 `PydanticOutputParser` 를 사용하겠습니다.

스키마를 준수하지 않는 결과를 전달하면 어떤 일이 벌어질지 확인해 주세요.


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [5]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [ ]:
parser.parse(misformatted)

이제 출력 고정 파서를 생성하고 사용할 수 있습니다.

이 출력 구문 분석기는 다른 출력 구문 분석기뿐만 아니라 형식 지정 오류를 수정하기 위한 LLM도 인수로 받습니다.


In [7]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [9]:
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [11]:
actor = new_parser.parse(misformatted)

In [14]:
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])